In [1]:
%matplotlib inline
#Codes by Shucheng Yang
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pycbc.psd

from pycbc.types.timeseries import TimeSeries
from pycbc.types.frequencyseries import FrequencySeries
from pycbc import filter


from scipy.interpolate import interp1d #导入scipy库
import os


def file2series(path):
    with open(path, 'r') as f:
        datafile = f.readlines()

    datafile = [np.fromstring(item.replace("D","e").replace('\n','').strip(), dtype= float, sep =' ') for item in datafile]
    datafile = np.array(datafile)
    ans = {}
    
    ans['timeVec'] = datafile[:,0]
    ans['n'] = len(ans['timeVec'])
    ans['sampIntrvl'] = ans['timeVec'][1]-ans['timeVec'][0]
    
    ans['hp'] = TimeSeries( datafile[:,5], delta_t=ans['sampIntrvl'], dtype = float,copy=True)
    ans['hc'] = TimeSeries( datafile[:,6], delta_t=ans['sampIntrvl'], dtype = float,copy=True)    
    return ans


#绘制波形图
def plotwave(hp,hc):
    plt.figure(figsize=(8,8/1.5))
    plt.plot(hp.sample_times, hp, label = '$h_{+}$')
    plt.plot(hc.sample_times, hc, label = '$h_{\\times}$')
    # plt.xlim(40,40.5)
    # plt.ylim(- 2e-22,2e-22)
    plt.xlabel("Time / s")
    plt.ylabel("$Strain$")
    plt.legend()
    plt.tight_layout()
    plt.grid(linestyle = "dotted", color = "#d3d3d3" , which="both")
    plt.show()


def stdf(h, f_max, freqIntrvl):
    #interp1d
    hf = h.to_frequencyseries()    
    freqVec = hf.sample_frequencies
    hf_value = np.array(hf)
    fit=interp1d(freqVec, hf_value ,fill_value = "extrapolate")  
    #
    stdFreqVec = np.arange(0, f_max, freqIntrvl)
    hf_value_std = fit(stdFreqVec)
    stdf_series = pycbc.types.frequencyseries.FrequencySeries(hf_value_std, delta_f=freqIntrvl, epoch='', copy=True)
    return stdf_series


def overlap_match_func(hf1, hf2, psd, f_min, f_max):
    roverlap = filter.matchedfilter.overlap(hf1, hf2, psd=psd, low_frequency_cutoff=f_min , high_frequency_cutoff=f_max, normalized=True)  
    amplitude1 = filter.matchedfilter.sigmasq(hf1, psd=psd, low_frequency_cutoff=f_min, high_frequency_cutoff=f_max )
    amplitude2 = filter.matchedfilter.sigmasq(hf2, psd=psd, low_frequency_cutoff=f_min, high_frequency_cutoff=f_max )
    rmatch,nn =filter.matchedfilter.match(hf1, hf2, psd=psd, low_frequency_cutoff=f_min , high_frequency_cutoff=f_max,v1_norm=True ,v2_norm=True)
    rmatch = rmatch / np.sqrt(amplitude1 * amplitude2)
    return roverlap, rmatch

In [2]:
path1 = "/home/ysc/Desktop/nw1/heco60_NS14_a60s50_R9_nu250_f10Hz_fisco.dat"
ans1 = file2series(path1)

hp1 = ans1["hp"]
hc1 = ans1["hc"]
timeVec1 = ans1["timeVec"]
n1 = len(hc1)
sampIntrvl1 = ans1["sampIntrvl"]
print(n1,sampIntrvl1)

path2 = "/home/ysc/Desktop/nw2/hbh60_NS14_a60_nu250_f10Hz_fisco.dat"
ans2 = file2series(path2)

hp2 = ans2["hp"]
hc2 = ans2["hc"]
timeVec2 = ans2["timeVec"]
n2 = ans2["n"]
sampIntrvl2 = ans2["sampIntrvl"]
print(n2,sampIntrvl1)

257022 0.000295529457
212102 0.000295529457


# 参数设置

In [3]:
assert sampIntrvl1 == sampIntrvl2 

sampIntrvl = sampIntrvl1
n_min = min(n1,n2)          #采样点数(Sampling Number), 有时也称为信号长度(Length of Signal) 2^16为2的幂时，快速傅里叶变化效率最高
                            #n =  duration * sampFreqint = (duration / sampIntrvl)
sampFreq = 1/sampIntrvl  #采样频率(Sampling frequency)，单位时间样本点个数，应大于 2f（即Nyquist频率)

duration = n_min/sampFreq #40        #信号持续时间(duration of signal) 2^16, 0.75d, 2^25 ,1.06yr

sampIntrvl = 1.0 / sampFreq                   #采样周期(Sampling period)，隔多少时间取样一次，或步长
freqIntrvl =  1 / (n_min * sampIntrvl)                     #傅里叶变换 频率分辨率(Frequency Interval) 
                                              # freqIntrvl = 1 / duration = 1 / (n * sampIntrvl)
                                              #            = sampFreq / n  
        
f_min = 20              #低于此频率的psd将被设置为0
f_max = sampFreq/2             #信号模式的最大频率

print("采样频率为%fHz，信号持续时间%ds, 时域信号采样%d 个点"%(sampFreq,duration,n_min))
print("信号中可分析最大频率为%fHz"%f_max)
print("\n采样周期，即时域分辨率为%fs"%(sampIntrvl))
print("信号频域的频率间隔，即频域分辨率为%fHz"%freqIntrvl);


# 列出lalsuite内置的解析psd (没发现有LISA的，下面以LIGO的作为示例)
# print(pycbc.psd.get_lalsim_psd_list())

#示例，psd参见， https://dcc.ligo.org/LIGO-T1800044/public
psd = pycbc.psd.from_string('aLIGOaLIGODesignSensitivityT1800044', n_min , freqIntrvl, f_min)

# #绘制 频率 - sqrt(PSD) 图
# plt.figure(figsize=(8,8/1.5))
# plt.loglog(psd.sample_frequencies, np.sqrt(psd), label = 'LIGO-psd')
# plt.xlim(2e1,1e3)
# plt.xlabel("Frequency / Hz")
# plt.ylabel("$\sqrt{S_{n}(f) \ / \ (Hz^{-1})}$")
# plt.legend()
# plt.tight_layout()
# plt.grid(linestyle = "dotted", color = "#d3d3d3" , which="both")
# plt.show()

# 波形图
# plotwave(hp2,hc2)

采样频率为3383.757444Hz，信号持续时间62s, 时域信号采样212102 个点
信号中可分析最大频率为1691.878722Hz

采样周期，即时域分辨率为0.000296s
信号频域的频率间隔，即频域分辨率为0.015953Hz


# match and overlap

In [4]:
hpf1 = stdf(hp1, f_max, freqIntrvl)
hpf2 = stdf(hp2, f_max, freqIntrvl)

hcf1 = stdf(hc1, f_max, freqIntrvl)
hcf2 = stdf(hc2, f_max, freqIntrvl)

overlap_hp, match_hp = overlap_match_func(hpf1, hpf2, psd, f_min, f_max)
overlap_hc, match_hc = overlap_match_func(hcf1, hcf2, psd, f_min, f_max)

print("Results:\nh+: match = {1:6.4f}, overlap = {0:6.4f}".format(overlap_hp, match_hp))
print("hx: match = {1:6.4f}, overlap = {0:6.4f} \n".format(overlap_hc, match_hc))

Results:
h+: match = 0.9939, overlap = 0.9704
hx: match = 0.9922, overlap = 0.9705 

